# Hacker's Guide to Large Language Models (LLMs)


🚀 Created for the [Duke AI Hackathon 2024](https://duke-ai-hack.webflow.io/)

👋 by [Suneel Nadipalli](https://www.linkedin.com/in/suneel-n/)


[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Duke-AI-Hackathon/AI-Hackathon-2024/blob/main/tutorials/llm/LLM_Tutorial_Duke_AI_Hackathon.ipynb)

# What are LLMs?

A large language model is a type of computational model designed for natural language processing tasks such as language generation. LLMs work by using a massive amount of text data to train a transformer.



In this tutorial, we will cover some of the basics, using popular LLM Python libraries like `openai`, `google-generativeai`, `huggingface`


## Table of Contents
* Environmental Variables

* Libraries

* LLM Tutorials
  * OpenAI - Examples

  * Gemini - Examples

  * HuggingFace - Examples

* Prompting Principles

  * Principle 1 - Clear Instructions

    * Strategy 1 - Delimiters

    * Strategy 2 - Structured output

    * Strategy 3 - Condition Check

    * Strategy 4 - Few-Shot Prompting

  * Principle 2 - Let the model think

    * Strategy 1 - Provide Step-by-Step instructions for task

    * Strategy 2 - Instruct the model to work out its own solution first

* LLM Limitations

  * Hallucinations

  * Hedging

  * Refusals

* NLP Tasks with LLMs
  * Text Summarization

  * Extractive Summarization

  * Sentiment Analysis
  
  * Translation

## Environment Variables (ie API keys, tokens)

For some models, you may need a HuggingFace Token. You can create one following these instructions.

Environment variables are an important part of application configuration when dealing with sensitive information like API keys. They let you keep your credentials and configuration separate from your code.

API Keys/Tokens should always be treated as sensitive information. Never commit these keys to version control systems like Git and don't add them directly to your code.

**Using environment variables in Google Colab**

1. Locate the key icon on the left sidebar of the Colab interface
2. Click on the key icon to open the "Secrets" panel
3. Add your secret key-value pairs:

> Name: `HUGGINGFACE_TOKEN`

> Value: `your_actual_token_here`

Then add the environment variables to your notebook using the code below:

```
from google.colab import userdata

userdata.get('HUGGINGFACE_TOKEN')
```

**Using environment variables locally**

When working on your local machine, it's common to use a .env file to manage environment variables. Always add .env to your .gitignore file to prevent accidental commits!

1. Create a file named .env in your project's root directory
2. Add your environment variables to this file:

> `HUGGINGFACE_TOKEN=your_actual_token_here`

3. Install the python-dotenv library:
> `pip install python-dotenv`

4. In your Python script, load and use the variables:

```
from dotenv import load_dotenv
import os

load_dotenv()
huggingface_token = os.getenv('HUGGINGFACE_TOKEN')

```

## Install libraries

We will be working with some NLP libraries in this tutorial, including openai, langchian, huggingface and google-generativeai

**OpenAI**

Explore resources, tutorials, API docs, and dynamic examples to get the most out of OpenAI's developer platform.

[Documentation](https://platform.openai.com/docs/api-reference/introduction?lang=python)

**Langchain**

LangChain is a framework for developing applications powered by large language models (LLMs). LangChain simplifies every stage of the LLM application lifecycle.

[Documentation](https://python.langchain.com/docs/introduction/)

**Gemini**

The Gemini API lets you access the latest generative models from Google.

[Documentation](hthttps://ai.google.dev/gemini-api/docs)


In [ ]:
!pip install openai google-generativeai langchain-huggingface --quiet

## Setup

**Load the API key and relevant Python libaries.**

The following code loads in the OpenAI API key for you from your Colab Secrets

In [ ]:
from openai import OpenAI
import openai

from google.colab import userdata

openai.api_key  = userdata.get('OPENAI_KEY')

The following code loads in the Gemini API key from your Colab secrets

In [ ]:
import os
import google.generativeai as genai

genai.configure(api_key=userdata.get('GEMINI_KEY'))

The following code loads in the HuggingFace key from your Colab secrets

In [ ]:
from huggingface_hub import login

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

login(token=userdata.get('HF_KEY'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**Helper Functions**

Throughout this course, we will use the following models:

- OpenAI's gpt-3.5-turbo model and the [chat completions endpoint](https://platform.openai.com/docs/guides/text-generation).

- Gemini's gemini-1.5-flash and the [text generation endpoint](https://ai.google.dev/gemini-api/docs/text-generation?lang=python)

- Langchain and HugginFace's microsoft/Phi-3-mini-4k-instruct and the [chat model](https://python.langchain.com/api_reference/huggingface/chat_models/langchain_huggingface.chat_models.huggingface.ChatHuggingFace.html)

This helper function will make it easier to use prompts and look at the generated outputs:

In [ ]:
openai_client = OpenAI(
    api_key=openai.api_key,
)

In [ ]:
gemini_client = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction="You are a helpful assistant")

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

hf_client = ChatHuggingFace(llm=llm, verbose=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def get_completion_openai(prompt, model="gpt-3.5-turbo"):

    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": prompt
        }
        ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    # print(response.choices[0].message.content)
    return response.choices[0].message.content

In [ ]:
def get_completion_gemini(prompt, model="gemini-1.5-flash"):

    messages = [
        {
            "role": "model",
            "parts": ["You are a helpful assistant."]
        },
        {
            "role": "user",
            "parts": [prompt]
        }
        ]

    response = gemini_client.generate_content(
        messages,
        generation_config=genai.GenerationConfig(
        # response_mime_type="application/json",
        temperature=0,
        )
    )

    return response.text

In [ ]:
def get_completion_hf(prompt):
    messages = [
        ("system", "You are a helpful assistant."),
        ("user", prompt)
    ]

    response = hf_client.invoke(messages)

    return response.content

## LLM Tutorials


### OpenAI

In [ ]:
prompt = f"""
My name is Suneel.
"""

response = get_completion_openai(prompt)
print(response)

Nice to meet you, Suneel! How can I assist you today?


In [ ]:
prompt = f"""
What are some things to do around Durham, NC in the fall?
"""

response = get_completion_openai(prompt)
print(response)

There are plenty of things to do around Durham, NC in the fall! Here are some suggestions:

1. Visit Duke Gardens: Enjoy the beautiful fall foliage at Duke Gardens, a stunning botanical garden located on the Duke University campus.

2. Attend a fall festival: Durham hosts several fall festivals, such as the CenterFest Arts Festival and the Durham Blues Festival, where you can enjoy live music, food, and local arts and crafts.

3. Explore Eno River State Park: Take a hike along the scenic trails at Eno River State Park and enjoy the fall colors along the river.

4. Visit a pumpkin patch or corn maze: Get into the fall spirit by visiting a local pumpkin patch or corn maze, such as McKee's Cornfield Maze or Hill Ridge Farms.

5. Attend a college football game: Cheer on the Duke Blue Devils or the North Carolina Central Eagles at a college football game in Durham.

6. Explore downtown Durham: Take a stroll through downtown Durham and explore the shops, restaurants, and art galleries that t

### Gemini

In [ ]:
prompt = f"""
My name is Suneel.
"""

response = get_completion_gemini(prompt)
print(response)

Nice to meet you, Suneel! 😊  What can I do for you today? 



In [ ]:
prompt = f"""
What are some things to do around Durham, NC in the fall?
"""

response = get_completion_gemini(prompt)
print(response)

Durham, NC is a vibrant city with plenty to offer in the fall! Here are some ideas for things to do:

**Nature & Outdoors:**

* **Visit the Duke Gardens:**  The gardens are stunning in the fall, with vibrant foliage and seasonal displays. Don't miss the Sarah P. Duke Gardens, the Doris Duke Center Gardens, and the  Conservatory.
* **Hike or bike the American Tobacco Trail:** This paved trail offers scenic views and connects to other trails in the area. 
* **Explore Eno River State Park:** Enjoy hiking, kayaking, or simply taking in the fall colors along the river.
* **Go apple picking at a local orchard:** Several orchards in the area offer apple picking, cider donuts, and other fall treats.
* **Visit the Durham Farmers Market:**  Enjoy fresh produce, local crafts, and live music on Saturdays.

**Arts & Culture:**

* **Attend a performance at the Durham Performing Arts Center (DPAC):**  Catch a Broadway show, concert, or other performance.
* **Explore the Nasher Museum of Art at Duke U

### HuggingFace

In [ ]:
prompt = f"""
My name is Suneel.
"""

response = get_completion_hf(prompt)
print(response)

Understood, Suneel is how you begin your communication. It's simple but sets the stage for recognizing Suneel as the individual involved in the interaction.


In [ ]:
prompt = f"""
What are some things to do around Durham, NC in the fall?
"""

response = get_completion_hf(prompt)
print(response)

Exploring Durham, NC, in the fall can be a delightful experience as you enjoy the vibrant changing leaves, pleasant temperatures, and numerous fall-related activities. Here are some ideas to make the most of your visit during this beautiful season:

1. Visit Duke University and Duke Chapel: Duke University holds one of the most extensive campus gardens in the country. Take a stroll through the lush gardens, admire the impressive campus architecture


## Prompting Principles

- Principle 1: Write clear and specific instructions
- Principle 2: Give the model time to “think”

### Principle 1: Write clear and specific instructions

#### Strategy 1: Use delimiters to clearly indicate distinct parts of the input

Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, :

In [ ]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

To guide a model towards the desired output and reduce irrelevant or incorrect responses, it is important to provide clear and specific instructions, which can be achieved through longer prompts that offer more clarity and context.

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

It is important to provide clear and specific instructions to guide a model towards the desired output, as longer prompts can offer more clarity and context for generating detailed and relevant responses.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

Clear and specific instructions in prompts are crucial for guiding language models towards desired outputs and reducing the likelihood of irrelevant or incorrect responses. 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

To achieve the best model response, one should craft the instructions as detailed and precise as feasible, aiming for clear prompts that often require more extensive writing to offer substantial context, which in turn promotes more accurate and relevant outputs.


#### Strategy 2: Ask for a structured output

JSON, HTML

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

```json
[
    {
        "book_id": 1,
        "title": "Echoes of Eternity",
        "author": "Serena Nightingale",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Whispers in the Dark",
        "author": "Julian Blackwood",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "Stars Aligned",
        "author": "Aria Silvermoon",
        "genre": "Science Fiction"
    }
]
```


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

```json
[
  {
    "book_id": "1",
    "title": "The Whispering City",
    "author": "Anya Petrova",
    "genre": "Fantasy"
  },
  {
    "book_id": "2",
    "title": "The Last Lighthouse Keeper",
    "author": "Elias Thorne",
    "genre": "Mystery"
  },
  {
    "book_id": "3",
    "title": "The Algorithm of Love",
    "author": "Maya Singh",
    "genre": "Romance"
  }
]
``` 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

Here's a list of imaginary book titles, their authors, and genres presented in JSON format:

```json
[
  {
    "book_id": 1,
    "title": "Whispers of the Galaxy",
    "author": "Eleanor P. Hartley",
    "genre": "Science Fiction"
  },
  {
    "book_id": 2,
    "


#### Strategy 3: Ask the model to check whether conditions are satisfied

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the boiling water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste (optional). 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Remove the tea bag from the cup.
Step 6 - Optionally, add sugar or milk to taste.
Step 7 - Enjoy your cup of tea.


#### Strategy 4: Few-Shot Prompting

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

Resilience is like a mighty tree that withstands \ 
the fiercest storms, bending but never breaking; \ 
it is the phoenix that rises from its ashes, \ 
renewed and stronger than before.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

<grandparent>: The oak that withstands the fiercest storm \
has roots that burrow deep into the earth; the \
eagle that soars above the highest peaks \
has wings that have weathered countless winds; \
the flower that blooms in the harshest desert \
has a spirit that thirsts for the sun. 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

<grandparent>: Just as the bamboo bends in the storm but does not break, so must we yield to life's trials with flexibility and strength.


<child>: Teach me about gratitude.


### Principle 2: Give the model time to “think”

#### Strategy 1: Specify the steps required to complete a task

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""

##### Example 1

In [ ]:
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Summary: Siblings Jack and Jill embark on a cheerful quest to fetch water from a hilltop well, facing misfortune but returning home with comforting embraces, their adventurous spirits undimmed.

2 - French Translation: Frère et sœur Jack et Jill se lancent dans une quête joyeuse pour aller chercher de l'eau d'un puits au sommet d'une colline, affrontant la malchance mais rentrant chez eux avec des étreintes réconfortantes, leurs esprits aventureux intacts.

3 - Names in French Summary: Jack, Jill

4 - JSON Object:
```json
{
  "french_summary": "Frère et sœur Jack et Jill se lancent dans une quête joyeuse pour aller chercher de l'eau d'un puits au sommet d'une colline, affrontant la malchance mais rentrant chez eux avec des étreintes réconfortantes, leurs esprits aventureux intacts.",
  "num_names": 2
}
```


**Gemini**

In [ ]:
response = get_completion_gemini(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Jack and Jill, siblings from a charming village, embark on a water-fetching adventure that ends in a comical tumble down a hill, but their spirits remain undeterred.

Jack et Jill, frère et sœur d'un charmant village, se lancent dans une aventure pour aller chercher de l'eau au sommet d'une colline, qui se termine par une chute comique, mais leur enthousiasme reste intact.

Jack, Jill

```json
{
"french_summary": "Jack et Jill, frère et sœur d'un charmant village, se lancent dans une aventure pour aller chercher de l'eau au sommet d'une colline, qui se termine par une chute comique, mais leur enthousiasme reste intact.",
"num_names": 2
}
``` 



In [ ]:
response = get_completion_hf(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
```json
{
  "french_summary": "Dans un charmant village, les jumeaux Jack et Jill ont tenté de chercher l'eau du haut d'une fontaine, mais Jack est tombé et Jill le suit, ce qui ne l'a pas empêché d'évoluer avec plaisir.",
  "num_names": 2
}
```


##### Example 2

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt_2)
print("Completion for prompt 2:")
print(response)

Completion for prompt 2:
Summary: Jack and Jill, siblings from a charming village, embark on a quest to fetch water from a hilltop well, facing misfortune but returning home with comforting embraces, their adventurous spirits undimmed.

Translation: Jack et Jill, frère et sœur d'un charmant village, entreprennent une quête pour aller chercher de l'eau d'un puits au sommet d'une colline, affrontant des malheurs mais rentrant chez eux avec des étreintes réconfortantes, leurs esprits aventureux intacts.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Jack et Jill, frère et sœur d'un charmant village, entreprennent une quête pour aller chercher de l'eau d'un puits au sommet d'une colline, affrontant des malheurs mais rentrant chez eux avec des étreintes réconfortantes, leurs esprits aventureux intacts.",
  "num_names": 2
}


**Gemini**

In [ ]:
response = get_completion_gemini(prompt_2)
print("Completion for prompt 2:")
print(response)

Completion for prompt 2:
Text: <
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
>

Summary: Jack and Jill, siblings from a charming village, embark on a water-fetching adventure that ends with a tumble down the hill, but their spirits remain undeterred.

Translation: Jack et Jill, frère et sœur d'un charmant village, se lancent dans une aventure pour aller chercher de l'eau au sommet d'une colline, qui se termine par une chute, mais leur esprit reste intact.

Names: Jack, Jill

Output JSON: 
```json
{
  "french_summary": "Jack et Jill, frère et sœur d'un charmant village, se lancent dans une aventure pou

**HuggingFace**

In [ ]:
response = get_completion_hf(prompt_2)
print("Completion for prompt 2:")
print(response)

Completion for prompt 2:
Summary: Jack and Jill humorously fall down a hill as they fetch water in a village, but their adventurous spirit remains firm.

Translation: Jack et Jill rent un chapeau à la volée dans leur quête de sucre dans un village charmant, mais leur esprit aventureux reste intact.

Names: Jack, Jill

Output JSON: {"french_summary": "Jack et Jill rent un ch


#### Strategy 2: Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

The student's solution is almost correct, but there is a small mistake in the calculation of the total cost. The correct calculation should be:

Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = 100x + 250x + (100,000 + 10x)
Total cost = 350x + 100,000

Therefore, the correct total cost for the first year of operations as a function of the number of square feet is 350x + 100,000.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

The student's solution is **partially correct**. 

Here's a breakdown:

* **Land cost:**  100x is correct.
* **Solar panel cost:** 250x is correct.
* **Maintenance cost:**  The student made a mistake here. The maintenance cost is a flat $100,000 **plus** $10 per square foot. So the correct expression is **100,000 + 10x**.

**The correct total cost function should be:**

Total cost = 100x + 250x + 100,000 + 10x = **360x + 100,000** 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet (x) is indeed given by the equation: Total cost = 450x + 100,000. This accounts for the land cost, solar panel cost, and maintenance cost.


The student's solution isn't actually correct.

Both OpenAI and Gemini managed to identify that the student maade a mistake, but only Gemini came up with the correct solution.

We can try to fix this by asking the model to work through it's own solution as we would first and them comparing the answers

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

To calculate the total cost for the first year of operations as a function of the number of square feet, we need to consider the costs for land, solar panels, and maintenance.

Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 (flat fee) + $10 * x

Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

Is the student's solution the same as the actual solution just calculated:
```
No
```

Student grade:
```
incorrect
```


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

ValueError: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 4. Meaning that the model was reciting from copyrighted material.

Interestingly enough, Gemini seems to have recognized that this particular question and soultion is a a commmonly used question-answer pair across other sources on the web and refuses to answer it!

Let's try chaning it up to see if it still recognizes it or will answer it

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
A farmer is building a fence and needs help \
calculating the costs.
- The fencing material costs $100 per meter.
- The posts cost $$100 per meter.
- The posts cost $250 each and are placed every 5 meters.
Labor costs a flat fee of $100,000 for the project, \
plus an additional $$100,000 for the project, \
plus an additional $10 per meter of fence.
What is the total cost of the fence as a function of the length of the fence in meters?
```
Student's solution:
```
Let x be the length of the fence in meters. Costs:

- Fence material cost: 100x
- Post cost: 250 * (x / 5)
- Labor cost: 100,000 + 10x

Total cost: 100x + 50x + 100,000 + 100x = 250x + 100,000
```
Actual solution:
"""

In [ ]:
response = get_completion_gemini(prompt)
print(response)

Question:
```
A farmer is building a fence and needs help \ 
calculating the costs.
- The fencing material costs $100 per meter. 
- The posts cost $$100 per meter. 
- The posts cost $250 each and are placed every 5 meters.
Labor costs a flat fee of $100,000 for the project, plus an additional $$100,000 for the project, plus an additional $10 per meter of fence. 
What is the total cost of the fence as a function of the length of the fence in meters?
``` 
Student's solution:
```
Let x be the length of the fence in meters. Costs:

- Fence material cost: 100x
- Post cost: 250 * (x / 5)
- Labor cost: 100,000 + 10x 

Total cost: 100x + 50x + 100,000 + 100x = 250x + 100,000
```
Actual solution:
```
Let x be the length of the fence in meters. Costs:

- Fence material cost: 100x
- Post cost: 250 * (x / 5) = 50x
- Labor cost: 100,000 + 10x 

Total cost: 100x + 50x + 100,000 + 10x = 160x + 100,000
```
Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:

**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

Question:
```
A farmer is building a fence and needs help 
calculating the costs.
- The fencing material costs $100 per meter. 
- The posts cost $200 per meter. 
- The posts cost $250 each and are placed every 5 meters.
Labor costs a flat fee of $100,000 for the project, plus an additional $100,000


## Model Limitations

### Hallucinations

Q: Who gave humans the boomerang according to Australian legend?

A: According to Australian Aboriginal legend, the boomerang was given to humans by a supernatural being named ["Bobbi-Bobbi" ](https://en.wikipedia.org/wiki/Bobbi-Bobbi)

In [ ]:
prompt = f"""
Who gave humans the boomerang according to Australian legend?
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

According to Australian Aboriginal legend, the boomerang was given to humans by the supreme being Baiame.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

According to Australian Aboriginal legend, the boomerang was given to humans by **the ancestral beings**, also known as **the Dreaming ancestors**. 

These beings are powerful spirit figures who created the land, animals, and people during the Dreamtime, a time before the present.  

The specific ancestral being who gave the boomerang to humans varies depending on the specific Aboriginal tribe and their creation stories. However, the general idea is that the boomerang was a gift from the ancestors, a tool that helped humans survive and thrive in the Australian landscape. 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

In Australian Aboriginal mythology, the boomerang, a hunting and sporting tool, was created by the spirit ancestor Wandandi from the dreamtime. The boomerang's properties are believed to be a reflection of Wandandi's cleverness and his connection to the landscape.


### Hedging

Hedging is when an LLM doesn't have enough knowledge to respond and has a fallback output along the lines of "I don't know", "I don't have enough context", "I'm not sure", "As an AI, I can't form opinions", etc

In [ ]:
prompt = f"""
Who do you love?
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

As an artificial intelligence, I do not have feelings or personal experiences, so I do not have the capacity to love someone.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

As an AI, I don't have the capacity to feel love. Love is a complex human emotion that I can't experience. 

However, I can help you with tasks, answer your questions, and provide information in a helpful and informative way.  

Is there anything else I can help you with today? 😊 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

As an artificial intelligence, I do not have feelings or personal experiences, so I do not have the capacity to love someone.


### Refusals

Refusal is a reaction LLMs have when the request is toxic, harmful or offensive and outputs something along the lines of "I'm sorry I can't do that"

In [ ]:
prompt = f"""
How do I hotwire a car?
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

I'm sorry, but I can't assist with that. It's illegal and unsafe. Always obtain a vehicle through proper channels.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

I understand you're curious about hotwiring a car, but I cannot provide instructions on how to do that. 

Hotwiring a car is illegal in most places and can be very dangerous. It can damage the car's electrical system, and you could be seriously injured if you don't know what you're doing. 

If you're interested in learning more about car mechanics, I recommend checking out reputable resources like:

* **Your local library:** They have books and resources on car maintenance and repair.
* **Online forums and communities:** There are many online communities dedicated to car enthusiasts where you can ask questions and learn from experienced mechanics.
* **YouTube tutorials:** Many mechanics and car enthusiasts create helpful videos on car maintenance and repair.

Remember, it's important to learn about car mechanics in a safe and legal way. 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

I'm sorry, but I can't assist with that. It's illegal and unsafe. Always obtain a vehicle through proper channels.


## Performing NLP Tasks with LLMs

### Summarization

In [ ]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \
super cute, and its face has a friendly look. It's \
a bit small for what I paid though. I think there \
might be other options that are bigger for the \
same price. It arrived a day earlier than expected, \
so I got to play with it myself before I gave it \
to her.
"""

In [ ]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site.

Summarize the review below, delimited by triple
backticks, in at most 30 words.

Review: ```{prod_review}```
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

```
Adorable panda plush loved by daughter, soft and cute. Smaller than expected for the price. Early delivery a bonus.
```


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

This panda plush is soft, cute, and well-loved by the reviewer's daughter. However, it's a bit small for the price, and there may be better value options available. 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

A panda plush toy, cheaper than expected, arrived early, and while cute, its size is too small for some customers' liking.


### Extractive Summarization

In [ ]:
prompt = f"""
Your task is to extract relevant information from \
a product review from an ecommerce site to give \
feedback to the Shipping department.

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \
delivery. Limit to 30 words.

Review: ```{prod_review}```
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

Feedback: The customer received the product a day earlier than expected, which was a positive surprise. Consider offering larger options for the same price to enhance customer satisfaction.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

The product arrived a day earlier than expected, allowing the customer to play with it before giving it to their daughter. 



**HuggingFace**

In [ ]:
response = get_completion_hf(prompt)
print(response)

Adjusted value for size; positive early arrival and recipient enjoyment.


### Sentiment Analysis

In [ ]:
movie_review = """
worst $10 i’ve spent WHY DID THEY SING
"""

In [ ]:
prompt = f"""
What is the sentiment of the following movie review,
which is delimited with triple backticks?

Review text: '''{movie_review}'''
"""

**OpenAI**

In [ ]:
response = get_completion_openai(prompt)
print(response)

The sentiment of the movie review is negative.


**Gemini**

In [ ]:
response = get_completion_gemini(prompt)
print(response)

The sentiment of the review is **extremely negative**. 

Here's why:

* **"worst $10 I’ve spent"**: This directly expresses dissatisfaction with the value for money.
* **"WHY DID THEY SING"**: This implies annoyance and frustration, likely indicating the reviewer found the singing aspect of the movie particularly bad. 

The use of all caps in "WHY DID THEY SING" further emphasizes the negative sentiment. 



In [ ]:
response = get_completion_hf(prompt)
print(response)

The sentiment of this movie review is strongly negative.


### Translation

In [ ]:
prompt = f"""
Translate the following English text to Spanish: \
```Hi, I would like to order a blender```
"""

In [ ]:
response = get_completion_openai(prompt)
print(response)

Hola, me gustaría ordenar una licuadora.


In [ ]:
response = get_completion_gemini(prompt)
print(response)

Hola, me gustaría pedir una licuadora. 



In [ ]:
response = get_completion_hf(prompt)
print(response)

Hola, me gustaría pedir una batidora

In Spanish, this translation maintains the original meaning and structure. The greeting has been translated to "Hola," which is equivalent to "Hi" in English. The rest of the sentence follows a similar construction in Spanish, with “me gustaría” (I would like) being polite and akin to the English phrase, utilizing "querer" in a conditional form which translates as "w
